<a href="https://colab.research.google.com/github/pabloac31/TFG/blob/master/cifar10_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! git clone https://github.com/pabloac31/TFG.git
%cd TFG

In [0]:
%tensorflow_version 1.x

In [0]:
from cifar10_models import *
from utils_cifar10 import *
from adversarial_attacks import *

In [0]:
iv3 = inception_v3(pretrained=True)

In [0]:
# Define what device we are using
use_cuda=True
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

In [0]:
test_loader = testloader('./data', 256)

In [0]:
test_model(iv3, device, test_loader)

In [0]:
adv_loader = testloader('./data', 1)

In [0]:
test_fgsm('./img1.png', 0.56)

In [0]:
acc, adv_examples = test_fgsm(iv3, device, adv_loader, 0.007, iters=1)